# Talking to LDAP servers

To talk to LDAP servers (AD, LDAPS) we depend on the `ldap3` module which needs to be installed first:

In [ ]:
!pip install --upgrade --break-system-packages ldap3

## connect and bind to LDAP

**Note:** To connect and bind to our Active Directory, you need to be within the ETH domain.

The connection needs information about the server name. Make sure you pass the `use_ssl=True` parameter to ensure your password is not sent over the internet unencrypted during the bind:

In [ ]:
from ldap3 import Server, Connection, ALL, NTLM, SUBTREE, Tls

server = Server("d.ethz.ch", use_ssl=True)
username = input("Username: ")

Now, we can establish a connection. It happens in two steps:

1. connection to the server
2. bind – the actual authentication against the LDAPS server

Most LDAP Severs expect a **distinguished name** (DN) instead of just a username. Within a directory, a username is not necessarily unique (AD is an exception: the sAMAccountName is forced to be unique within the whole directory). 

If the `conn.bind()` is False, then the connection could not be established, e.g. wrong credentials or username (DN)

In [ ]:
import getpass
from ldap3 import Server, Connection, ALL, NTLM, SUBTREE, Tls

server = Server("d.ethz.ch", use_ssl=True)
username = input("Username:")

conn = Connection(
    server, 
    user=f'cn={username},OU=ETHUsers,DC=d,DC=ethz,DC=ch',
    password=getpass.getpass()
)
conn.bind()

**Note:** AD supports NTLM authentication (a proprietary protocol) to bind with `domain\username` instead of the usual distinguishedName (dn), so you don't need to know the exact distinguishedName:

In [ ]:
conn = Connection(
    server, 
    user=f'd\\{username}',
    password=getpass.getpass()
)
conn.bind()

## Searching

Once we got a successful bind, we can **start searching** for data

In [ ]:
attributes=["displayName", "mail"]
ok = conn.search(
    search_base="DC=d,DC=ethz,DC=ch",
    search_filter=f"(&(objectClass=user)(sAMAccountName={username}))",
    search_scope=SUBTREE,
    attributes=attributes
)
if ok:
    for response in conn.response:
        if not 'dn' in response: continue
        print(response['dn'])
        for key in response.get('attributes', []):
            print(key, ":", response['attributes'].get(key))

## Exercises

1. Query for groups, using `(&(objectClass=group)(sAMAccountName=id-sis*))`
2. Use the `*` to match any character (only use it at the end, for performance!)
3. Try to use `attributes=["*"]` to get all attributes (watch out, it can be a lot ;-)
4. Query for all computers starting with `Q`, using `(&(objectClass=computer)(sAMAccountName=q*))`, 